# US Immigration Analysis
### Data Engineering Capstone Project

#### Project Summary
In this project I am going to utilize I94 immmigration dataset and US demographic data to analyze the non immigrants travel patterns. Then I am going to create ETL pipeline using Spark to build a data warehouse using a star table schema for further downstream analysis. I will save results in a Parquet file which could be loaded to S3 or a Redshift cluster if needed. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import pandas as pd
from datetime import datetime
import datetime as dt

import os
import boto3

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, LongType as Long

from pyspark.sql.functions import udf, date_format, split, col, first, upper

get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project I created a data model to analyze the data and discover any relationship patterns between non-immigrant tourists and other metadata available such as demographic information with different preferences in their travel. 

I am going to use Spark to load and clean the data, parquet files to store outputs, and create a data model using star schema for data warehouse tables.

#### Describe and Gather Data 

### I94 Immigration Data: 
This data comes from the US National Tourism and Trade Office. [This](https://travel.trade.gov/research/reports/i94/historical/2016.html) is where the data comes from. 
Some of the important columns are 
- 1) i94Bir 	 Age of non-immigrant in years
- 2) i94visa 	 Visa codes collapsed into three categories
- 3) count 	 Used for summary statistics
- 4) i94mode 	 Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)
- 5) i94addr 	 USA State of arrival
- 6) visatype 	 Class of admission legally admitting the non-immigrant to temporarily stay in U.S.

### U.S. City Demographic Data: 
This data comes from OpenSoft. More details about it can be found [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
Some important columns are 
- 1) City
- 2) Median Age (overall median age within the city population)
- 3) Total Population
- 4) Male Population
- 5) Female Population
- 6) Count (number of people under specific race category anotated by Race column)


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [2]:
# Read in the data here
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()


In [3]:
us_immg_df=spark.read.parquet("sas_data")
us_immg_df.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [4]:
us_immg_df.select("i94res","i94port","arrdate","i94mode","depdate","i94bir","i94visa","count","gender",col("admnum").cast(Long())).show(10)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
| 438.0|    LOS|20574.0|    1.0|20582.0|  40.0|    1.0|  1.0|     F|94953870030|
| 438.0|    LOS|20574.0|    1.0|20591.0|  32.0|    1.0|  1.0|     F|94955622830|
| 438.0|    LOS|20574.0|    1.0|20582.0|  29.0|    1.0|  1.0|     M|94956406530|
| 438.0|    LOS|20574.0|    1.0|20588.0|  29.0|    1.0|  1.0|     F|94956451430|
| 438.0|    LOS|20574.0|    1.0|20588.0|  28.0|    1.0|  1.0|     M|94956388130|
| 464.0|    HHW|20574.0|    1.0|20579.0|  57.0|    2.0|  1.0|     M|94981802830|
| 464.0|    HHW|20574.0|    1.0|20586.0|  66.0|    2.0|  1.0|     F|94979689930|
| 464.0|    HHW|20574.0|    1.0|20586.0|  41.0|    2.0|  1.0|     F|94979746730|
| 464.0|    HOU|20574.0|    1.0|20581.0|  27.0|    2.0|  1.0|     M|94973246630|
| 464.0|    LOS|20574.0|    

### Clean immigration data
- Added a few static list for better readability and dropping duplicates


In [5]:
us_immg_df=us_immg_df.select(col("i94res").cast(Int()),col("i94port"),
                           col("arrdate").cast(Int()), \
                           col("i94mode").cast(Int()),col("depdate").cast(Int()),
                           col("i94bir").cast(Int()),col("i94visa").cast(Int()), 
                           col("count").cast(Int()), \
                           "gender",col("admnum").cast(Long()))
us_immg_df.show(5)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|   438|    LOS|  20574|      1|  20582|    40|      1|    1|     F|94953870030|
|   438|    LOS|  20574|      1|  20591|    32|      1|    1|     F|94955622830|
|   438|    LOS|  20574|      1|  20582|    29|      1|    1|     M|94956406530|
|   438|    LOS|  20574|      1|  20588|    29|      1|    1|     F|94956451430|
|   438|    LOS|  20574|      1|  20588|    28|      1|    1|     M|94956388130|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
only showing top 5 rows



In [6]:
us_immg_df.count(), us_immg_df.dropDuplicates().count()

(3096313, 3096302)

In [7]:
us_immg_df.dropDuplicates(['admnum']).count()
us_immg_df=us_immg_df.dropDuplicates()

In [8]:
travel_mode =[[1,'Air'],[2,'Sea'],[3,'Land'],[9,'Not Reported']]
travel_mode=spark.createDataFrame(travel_mode)
travel_mode.write.mode("overwrite").parquet('travel_mode.parquet')

In [9]:
visa_type = [[1, 'Business'], [2, 'Pleasure'], [3, 'Student']]
visa_type=spark.createDataFrame(visa_type)
visa_type.write.mode('overwrite').parquet('visa_type.parquet')

I used the I94_SAS_Labels_Descriptions.SAS file to extract the port information and residency information. I extracted those to text files which I used below.

In [10]:
ports_df = pd.read_csv('ports.txt',sep='=',names=['id','port'])
ports_df.head()
# Remove whitespaces and single quotes
ports_df['id']=ports_df['id'].str.strip().str.replace("'",'')

ports_df['port_city'], ports_df['port_state']=ports_df['port'].str.strip().str.replace("'",'').str.strip().str.split(',',1).str

ports_df['port_state']=ports_df['port_state'].str.strip()

ports_df.drop(columns =['port'], inplace = True)

ports_data=ports_df.values.tolist()
ports_schema = R([
    Fld('id', Str(), True),
    Fld('port_city', Str(), True),
    Fld('port_state', Str(), True)
])
ports=spark.createDataFrame(ports_data, ports_schema)
ports.show(5)
ports.write.mode('overwrite').parquet('./data/ports.parquet')


+---+--------------------+----------+
| id|           port_city|port_state|
+---+--------------------+----------+
|ALC|               ALCAN|        AK|
|ANC|           ANCHORAGE|        AK|
|BAR|BAKER AAF - BAKER...|        AK|
|DAC|       DALTONS CACHE|        AK|
|PIZ|DEW STATION PT LA...|        AK|
+---+--------------------+----------+
only showing top 5 rows



In [11]:
cities = pd.read_csv('residence_city.txt',sep='=',names=['id','country'])
cities['country']=cities['country'].str.replace("'",'').str.strip()
cities_data=cities.values.tolist()
cities_schema = R([
    Fld('id', Str(), True),
    Fld('country', Str(), True)
])
cities=spark.createDataFrame(cities_data, cities_schema)
cities.show(5)
cities.write.mode('overwrite').parquet('resident_city.parquet')


+---+--------------------+
| id|             country|
+---+--------------------+
|582|MEXICO Air Sea, a...|
|236|         AFGHANISTAN|
|101|             ALBANIA|
|316|             ALGERIA|
|102|             ANDORRA|
+---+--------------------+
only showing top 5 rows



In [12]:
us_immg_df = us_immg_df.join(ports, us_immg_df.i94port==ports.id, how='left')
us_immg_df.show(5)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---+---------+----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum| id|port_city|port_state|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---+---------+----------+
|   110|    BGM|  20550|      1|  20556|    36|      1|    1|     F|92847893530|BGM|   BANGOR|        ME|
|   108|    BGM|  20569|      1|  20571|    43|      1|    1|     M|59234302533|BGM|   BANGOR|        ME|
|   129|    BGM|  20559|      1|  20584|    67|      1|    1|     M|93553405030|BGM|   BANGOR|        ME|
|   261|    BGM|  20568|      1|   null|     9|      1|    1|     M|94455571030|BGM|   BANGOR|        ME|
|   135|    BGM|  20564|      1|  20569|    30|      2|    1|     M|94033156330|BGM|   BANGOR|        ME|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---+---------+----------+
only showing top 5 rows



In [13]:
us_immg_df = us_immg_df.withColumn("arrival_date", get_date(us_immg_df.arrdate))
us_immg_df=us_immg_df.drop("id")
us_immg_df.show(5)


+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---------+----------+------------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|port_city|port_state|arrival_date|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---------+----------+------------+
|   110|    BGM|  20550|      1|  20556|    36|      1|    1|     F|92847893530|   BANGOR|        ME|  2016-04-06|
|   108|    BGM|  20569|      1|  20571|    43|      1|    1|     M|59234302533|   BANGOR|        ME|  2016-04-25|
|   129|    BGM|  20559|      1|  20584|    67|      1|    1|     M|93553405030|   BANGOR|        ME|  2016-04-15|
|   261|    BGM|  20568|      1|   null|     9|      1|    1|     M|94455571030|   BANGOR|        ME|  2016-04-24|
|   135|    BGM|  20564|      1|  20569|    30|      2|    1|     M|94033156330|   BANGOR|        ME|  2016-04-20|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+

In [14]:
timestamp=us_immg_df.select(col('arrdate').alias('arrival_sasdate'),
                                   col('arrival_date').alias('arrival_iso_date'),
                                   date_format('arrival_date', 'y').alias('arrival_year'), 
                                   date_format('arrival_date','M').alias('arrival_month'),
                                   date_format('arrival_date','E').alias('arrival_dayofweek'), 
                                   date_format('arrival_date', 'd').alias('arrival_day')).dropDuplicates()
us_immg_df.drop('arrival_date').write.mode("overwrite").parquet('us_immigration.parquet')

In [15]:
timestamp.createOrReplaceTempView("timestamp")
timestamp=spark.sql('''select arrival_sasdate,
                         arrival_iso_date,
                         arrival_month,
                         arrival_dayofweek,
                         arrival_year,
                         arrival_day,
                         CASE WHEN arrival_month IN (12, 1, 2) THEN 'winter' 
                                WHEN arrival_month IN (3, 4, 5) THEN 'spring' 
                                WHEN arrival_month IN (6, 7, 8) THEN 'summer' 
                                ELSE 'autumn' 
                         END AS date_season from timestamp''')


In [16]:
timestamp.write.mode("overwrite").partitionBy("arrival_year", "arrival_month").parquet('timestamp.parquet')


### Read and clean Demographics data

In [17]:
demographics_df=spark.read.csv("us-cities-demographics.csv", sep=';', header=True)
demographics_df.columns

['City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Number of Veterans',
 'Foreign-born',
 'Average Household Size',
 'State Code',
 'Race',
 'Count']

### Clean Demographics data
- Remove duplicates 

In [18]:
demographics_df.select("City","state","Median Age","male population","female population","total population", \
                  "foreign-born","Average Household Size").orderBy("city").show(5)

+-------+-----+----------+---------------+-----------------+----------------+------------+----------------------+
|   City|state|Median Age|male population|female population|total population|foreign-born|Average Household Size|
+-------+-----+----------+---------------+-----------------+----------------+------------+----------------------+
|Abilene|Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
+-------+-----+----------+---------------+-----------------+----------------+-----------

In [19]:
demographics_df.select("city","state code","Race","count").orderBy("city").show()
race_counts=(demographics_df.select("city","state code","Race","count")
    .groupby(demographics_df.City, "state code")
    .pivot("Race")
    .agg(first("Count")))
us_dem_data=demographics_df.drop(*["Number of Veterans","Race","Count"]).dropDuplicates()

+-------+----------+--------------------+------+
|   city|state code|                Race| count|
+-------+----------+--------------------+------+
|Abilene|        TX|American Indian a...|  1813|
|Abilene|        TX|  Hispanic or Latino| 33222|
|Abilene|        TX|               White| 95487|
|Abilene|        TX|               Asian|  2929|
|Abilene|        TX|Black or African-...| 14449|
|  Akron|        OH|               White|129192|
|  Akron|        OH|  Hispanic or Latino|  3684|
|  Akron|        OH|Black or African-...| 66551|
|  Akron|        OH|               Asian|  9033|
|  Akron|        OH|American Indian a...|  1845|
|Alafaya|        FL|  Hispanic or Latino| 34897|
|Alafaya|        FL|               Asian| 10336|
|Alafaya|        FL|               White| 63666|
|Alafaya|        FL|Black or African-...|  6577|
|Alameda|        CA|               White| 44232|
|Alameda|        CA|American Indian a...|  1329|
|Alameda|        CA|Black or African-...|  7364|
|Alameda|        CA|

In [20]:
us_dem_data.join(race_counts, ["city","state code"]).orderBy("city").show(5)

+-------+----------+----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|   City|State Code|     State|Median Age|Male Population|Female Population|Total Population|Foreign-born|Average Household Size|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+-------+----------+----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|Abilene|        TX|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|                             1813| 2929|                    14449|             33222| 95487|
|  Akron|        OH|      Ohio|      38.1|          96886|           100667|          197553|       10024|      

In [21]:
us_dem_data=us_dem_data.join(race_counts, ["city","state code"])
us_dem_data=us_dem_data.select('City', col('State Code').alias('State_Code'), 'State', col('Median Age').alias('MedianAge'),
     col('Male Population').alias('Males'), col('Female Population').alias('Females'), 
     col('Total Population').alias('TotalPop'), 'Foreign-born', 
     col('Average Household Size').alias('AvgHouseholdSize'),
     col('American Indian and Alaska Native').alias('NativePopulation'), 
     col('Asian').alias('AsianPopulation'), 
     col('Black or African-American').alias('BlackPopulation'), 
     col('Hispanic or Latino').alias('LatinoPopulation'), 
     col('White').alias('WhitePoplation'))

us_dem_data=us_dem_data.drop("state")
us_dem_data.show(5)

+---------------+----------+---------+------+-------+--------+------------+----------------+----------------+---------------+---------------+----------------+--------------+
|           City|State_Code|MedianAge| Males|Females|TotalPop|Foreign-born|AvgHouseholdSize|NativePopulation|AsianPopulation|BlackPopulation|LatinoPopulation|WhitePoplation|
+---------------+----------+---------+------+-------+--------+------------+----------------+----------------+---------------+---------------+----------------+--------------+
|Highlands Ranch|        CO|     39.6| 49186|  53281|  102467|        8827|            2.72|            1480|           5650|           1779|            8393|         94499|
|           Kent|        WA|     33.4| 61825|  65137|  126962|       38175|            3.06|            3651|          26168|          20450|           21928|         67918|
|        Madison|        WI|     30.7|122596| 126360|  248956|       30090|            2.23|            2296|          23937|     

In [22]:
#write to parquet
us_dem_data.write.mode('overwrite').parquet("us-cities-demographics.parquet")

In [23]:
us_immg_df=us_immg_df.join(us_dem_data, (upper(us_immg_df.port_city)==upper(us_dem_data.City)) & \
                                           (upper(us_immg_df.port_state)==upper(us_dem_data.State_Code)), how='left')
us_immg_df.count()
us_immg_df=us_immg_df.drop("City","State_Code")


In [28]:
us_immg_df.count()
us_immg_df.show(5)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---------+----------+------------+---------+-----+-------+--------+------------+----------------+----------------+---------------+---------------+----------------+--------------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|port_city|port_state|arrival_date|MedianAge|Males|Females|TotalPop|Foreign-born|AvgHouseholdSize|NativePopulation|AsianPopulation|BlackPopulation|LatinoPopulation|WhitePoplation|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+---------+----------+------------+---------+-----+-------+--------+------------+----------------+----------------+---------------+---------------+----------------+--------------+
|   103|    NEC|  20556|      3|  20557|    51|      2|    1|     F|  788711085|    NECHE|        ND|  2016-04-12|     null| null|   null|    null|        null|            null|            null|           null|           nu

In [26]:
us_immg_df.drop('arrival_date').write.mode("overwrite").parquet('i94-immigration-data.parquet')


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

| Table | Type | Description | Columns | 
| --- | --- | --- | --- | 
| i94-immigration-data | Fact Table | Stores all the count information about imigration data | i94res, i94port, arrdate, i94mode, depdate, i94bir, i94visa, count, gender, admnum, port_city, port_state, arrival_date, MedianAge, Males, Females, TotalPop, Foreign-born, AvgHouseholdSize, NativePopulation, AsianPopulation, BlackPopulation, LatinoPopulation, WhitePoplation |
| resident_city |   Dimension Table | Contains info about resident city of the immigrant | id, country |
| ports | Dimension Table  | Port of entry information | id, city, port  |
| timestamp |  Dimension Table  | Arrival information |  arrival_sasdate, arrival_iso_date, arrival_month, arrival_dayofweek, arrival_year,arrival_day | 
| visa |  Dimension Table  | id | id, type | 
| travel_mode |  Dimension Table  | different travel modes | id, transport | 


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
- Load I94 immigation data set and dedupe and drop nulls values. 
- Select necessary columns for final analysis and clean null values 
- Build visa type and tranporation mode data frame in spark
- Read port of entry data set and residence country statis data from SAS file
- Join port of entry data frame and drop the duplicate id columns
- Read demographics data set and clean up duplicate and null values
- Aggregate results for different races and store them
- Join with port of entry data and eventually with immigration data frame. This will be the fact table
- Create timestamp dimension table from immigration data frame

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

- Created and etl.py file to do the ETL process.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [27]:
# Perform quality checks here
if us_dem_data.count() > 0:
    print('Deomographics data successfully persisted')
else:
    print('Deomographics data could not be saved!')

if us_dem_data.count() == race_counts.count():
    print('Aggregations were successful.')
else:
    print('Failed while aggregating data')

if us_immg_df.count() > 0:
    print('Immigration data saved successfully')
else:
    print('No data in immigration data frame')
    

Deomographics data successfully persisted
Aggregations were successful.
Immigration data saved successfully


#### 4.3 Data dictionary 

##### timestamp
- PK: arrival_sasdate ( arrival date)
- Transformed columns such as arrival_month, year, day, season from the above column

##### ports
- PK: id(port_id)
- city: Destination city

##### visa
- PK: id (visa_id)
- type: visa type 

##### resident_city
- PK: id (mode_id)
- country

##### travel_mode
- PK: id (mode_id)
- transport: transport type 

##### i94-immigration-data
- PK: admnum (Admission Number)

- arrdate (arrival date) FK : timestamp.arrival_sasdate
- depdate (Departure Date)
- i94port (port id) FK : ports.id
- i94res (resident_city) FK : resident_city.id
- i94mode (travel mode) FK : travel_mode.id
- i94visa (visa id) FK : visa.id
- arrdate (arrival date) FK : timestamp.arrival_sasdate
- age (age)
- gender 

--- City level aggregations 
- MedianAge (Median age of the people in the city)
- Males (Total male poppulation in the city)
- Females (Total female population in the city) 
- TotalPop
- Foreign-born
- AvgHouseholdSize 

--- Race level information about the city. Population of different races based on demographic information
- NativePopulation
- AsianPopulation 
- BlackPopulation
- LatinoPopulation
- WhitePoplation 




#### Step 5: Complete Project Write Up
##### Clearly state the rationale for the choice of tools and technologies for the project.
I used Pandas and Seaborn for analysis of smaller files. Whereas for larger files Spark was used. Apart from providing fast and in-memory processing of large datasets Spark also provided support to read SAS files as the immigration data was in this format. Eventually I stored the data in parquet file and sent it to S3 where it could be loaded into Redshift for further analysis.

##### Propose how often the data should be updated and why ? 
There are 2 types of data we dealt with. The demographics and Airport data is more static and won't change much. So we could load this once a month or so. They could be also updated manually if any new data is received. Where are the temperature and immigration data is more dynamic and can be updated daily/weekly or monthly based on downstream requirements. This would also depend on the frequency at which the source datasets are updated. 

##### Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 --  We could set up a EMR cluster in AWS and increase the number of nodes to handle the increased data. We could also used cluster manager such as Yarn to manage the resources for us.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 -- Airflow could be used to schedule different steps in the data pipeline. We could setup retries or integrate notification services with pagerduty if any of the steps fails. 
 * The database needed to be accessed by 100+ people.
 -- The parquet files could be loaded into Amazon Redshift cluster in order to achive high performance. Redshift cluster could be optimized to handle massive parallel requests.